In [12]:
data_dir = 'E:/Jupyter NB/10 class Classification/'

train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

# Defining Model

In [13]:
import numpy as np
import sys
sys.path.append('utils/')

import keras
from keras import layers
from keras import models

from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Lambda, concatenate, Activation
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras import backend as K

In [14]:
def preprocess_input(x):
    x /= 255.0
    x -= 0.5
    x *= 2.0
    return x

In [15]:
std_model = models.Sequential([
    layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (299,299,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(20, activation = 'softmax')
    
])

std_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 297, 297, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 148, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 146, 146, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 170528)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                10913856  
_________________________________________________________________
dense_4 (Dense)              (None, 20)               

# Model

In [16]:
data_generator = ImageDataGenerator(data_format='channels_last', preprocessing_function = preprocess_input)
#val_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)

validation_generator = data_generator.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)

'''
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20)
'''

Found 2000 images belonging to 20 classes.
Found 400 images belonging to 20 classes.


'\ntest_generator = data_generator.flow_from_directory(\n    test_dir,\n    target_size=(299, 299),\n    batch_size=20)\n'

In [17]:
def loss(y_true, y_pred):
    entropy = -K.mean(K.sum(y_pred*K.log(y_pred), 1))
    beta = 0.1
    return logloss(y_true, y_pred) - beta*entropy

In [18]:
from keras import optimizers

std_model.compile(loss=loss,
    optimizer=optimizers.SGD(lr=1e-3),
    metrics=['categorical_crossentropy', 'accuracy'])

In [ ]:
history = std_model.fit_generator(
    train_generator,
    steps_per_epoch=150,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=40)

Epoch 1/20
150/150 [==============================] - 22s 144ms/step - loss: 2.7263 - categorical_crossentropy: 3.0249 - accuracy: 0.0313 - val_loss: 2.6976 - val_categorical_crossentropy: 2.9958 - val_accuracy: 0.0575
Epoch 2/20
150/150 [==============================] - 23s 154ms/step - loss: 2.7024 - categorical_crossentropy: 3.0015 - accuracy: 0.0443 - val_loss: 2.8237 - val_categorical_crossentropy: 3.0227 - val_accuracy: 0.0525
Epoch 3/20
150/150 [==============================] - 28s 190ms/step - loss: 2.7095 - categorical_crossentropy: 3.0072 - accuracy: 0.0593 - val_loss: 2.7331 - val_categorical_crossentropy: 2.9689 - val_accuracy: 0.0425
Epoch 4/20
150/150 [==============================] - 29s 192ms/step - loss: 2.6552 - categorical_crossentropy: 2.9490 - accuracy: 0.0907 - val_loss: 2.9079 - val_categorical_crossentropy: 2.9063 - val_accuracy: 0.0850
Epoch 5/20
150/150 [==============================] - 29s 192ms/step - loss: 2.6130 - categorical_crossentropy: 2.9042 - acc